In [2]:
#!/usr/bin/env python3
"""
Count files in every sub‑directory under a given root and report the total.

Usage (from the command line):
    python count_files.py /path/to/root
"""

from pathlib import Path
import os
import sys
from pprint import pprint


def count_files_by_directory(root: str | os.PathLike = ".") -> tuple[dict[Path, int], int]:
    """
    Recursively count files inside every directory under *root*.

    Parameters
    ----------
    root : str | Path
        Directory to start walking from.

    Returns
    -------
    dir_counts : dict[Path, int]
        Mapping of absolute directory path → number of (non‑hidden) files it contains.
    total_files : int
        Sum of all counted files.
    """
    dir_counts: dict[Path, int] = {}
    total_files = 0
    root_path = Path(root).expanduser().resolve()

    for dirpath, _, filenames in os.walk(root_path):
        # Skip hidden files (starting with "."); remove this filter if you want to include them
        visible_files = [f for f in filenames if not f.startswith(".")]
        count = len(visible_files)

        dir_counts[Path(dirpath)] = count
        total_files += count

    return dir_counts, total_files


counts, total = count_files_by_directory()

pprint(counts)            # Nicely formats the per‑directory counts
print(f"\nTotal files: {total}")


{WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt'): 5,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git'): 7,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/branches'): 0,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/hooks'): 14,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/info'): 1,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/logs'): 1,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/logs/refs'): 0,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/logs/refs/heads'): 1,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/logs/refs/remotes'): 0,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/logs/refs/remotes/origin'): 1,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_service_attempt/.git/objects'): 0,
 WindowsPath('//wsl.localhost/Ubuntu/root/lecture_servi

In [3]:
#!/usr/bin/env python3
"""
Utility helpers:

- count_files_by_directory(root, include=None, exclude=None)
      → (dict[pathlib.Path, int], int)

- zip_tree(root, zip_path, include=None, exclude=None, *, keep_structure=True)
      → pathlib.Path  # the ZIP file that was written
"""

from pathlib import Path
import os
import zipfile
from typing import Iterable, Sequence


def _normalize_list(
    items: Sequence[str | os.PathLike] | None, root: Path
) -> set[Path] | None:
    """Return absolute Path objects or None if *items* is None/empty."""
    if not items:
        return None
    return {root.joinpath(Path(p).as_posix()).resolve() for p in items}


def _should_skip(path: Path, include: set[Path] | None, exclude: set[Path] | None) -> bool:
    """True if *path* (a directory) must be skipped according to include/exclude."""
    if include is not None:
        # Skip everything NOT under one of the include paths
        return not any(include_path in path.parents or include_path == path for include_path in include)
    if exclude is not None:
        # Skip anything that is (or is inside) an excluded dir
        return any(ex_path in path.parents or ex_path == path for ex_path in exclude)
    return False


def count_files_by_directory(
    root: str | os.PathLike = ".",
    *,
    include: Sequence[str | os.PathLike] | None = None,
    exclude: Sequence[str | os.PathLike] | None = None,
) -> tuple[dict[Path, int], int]:
    """
    Recursively count visible files in *root*, honoring optional include/exclude.

    include / exclude:
        Iterable of directory paths relative to *root*.
        • If *include* is given, ONLY those paths (and their sub‑trees) are considered.
        • If *include* is None but *exclude* is provided, those paths are skipped.
        • Leading dots (hidden dirs/files) are always ignored.
    """
    root = Path(root).expanduser().resolve()
    include_set = _normalize_list(include, root)
    exclude_set = _normalize_list(exclude, root)

    dir_counts: dict[Path, int] = {}
    total = 0

    for dirpath, _, filenames in os.walk(root):
        dir_path = Path(dirpath)

        # Determine if this directory should be processed
        if _should_skip(dir_path, include_set, exclude_set):
            continue

        # Skip hidden directories entirely
        if dir_path.name.startswith("."):
            continue

        visible_files = [f for f in filenames if not f.startswith(".")]
        count = len(visible_files)
        dir_counts[dir_path] = count
        total += count

    return dir_counts, total


def zip_tree(
    root: str | os.PathLike,
    zip_path: str | os.PathLike,
    include: Sequence[str | os.PathLike] | None = None,
    exclude: Sequence[str | os.PathLike] | None = None,
    *,
    keep_structure: bool = True,
) -> Path:
    """
    Compress *root* into *zip_path* applying the same include/exclude logic.

    Parameters
    ----------
    keep_structure : bool
        • True  → store each file with its path relative to *root* (default).  
        • False → store only basenames (may cause name clashes).

    Returns  -------
    Path to the created ZIP archive.
    """
    root = Path(root).expanduser().resolve()
    zip_path = Path(zip_path).expanduser().resolve()
    include_set = _normalize_list(include, root)
    exclude_set = _normalize_list(exclude, root)

    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for dirpath, _, filenames in os.walk(root):
            dir_path = Path(dirpath)

            if _should_skip(dir_path, include_set, exclude_set):
                continue
            if dir_path.name.startswith("."):
                continue

            for fname in filenames:
                if fname.startswith("."):
                    continue
                fpath = dir_path / fname
                arcname = fpath.relative_to(root) if keep_structure else fpath.name
                zf.write(fpath, arcname)

    return zip_path


# -------- example usage --------
if __name__ == "__main__":
    project_root = Path(".")  # current directory

    # Only include specific sub‑folders (relative to root)
    include_dirs = ["src", "tests"]

    # counts, total = count_files_by_directory(project_root, include=include_dirs)
    # print("Per‑directory counts:")
    # for d, c in counts.items():
    #     print(f"{d}: {c}")
    # print(f"TOTAL: {total}")

    # Create archive, excluding virtual‑env and build artifacts
    archive = zip_tree(
        project_root,
        "to_overleaf.zip",
        include=["latex-math", "slides", "style"],
    )
    print(f"\nCreated ZIP archive at {archive}")



Created ZIP archive at \\wsl.localhost\Ubuntu\root\lecture_service_attempt\to_overleaf.zip
